# Core
> Reusable utils for download caching and pipelining.

In [79]:
# | default_exp core


In [80]:
# | hide
from nbdev.showdoc import *


In [81]:
# | export


import dataclasses
import sys
from typing import Any, Callable, Union, List


import joblib
import joblib.memory
import requests as rq
from fastcore.all import *
from tqdm import tqdm


In [82]:
# | export


@dataclasses.dataclass
class DownloadContent:
    """
    Masks the repr with it's hash to avoid serialising the content.
    This stops libraries like joblib serialising large strings in input reprs
    """

    content: bytes

    def __repr__(self):
        return joblib.hash(self.content)


In [83]:
# | exporti


class Standin:
    def cache(self, func):
        return func


In [84]:
# | export

if __name__ != "__main__":
    if sys.platform == "linux":
        cache = joblib.Memory(
            "/mnt/d/.joblib", verbose=0, compress=True, bytes_limit=int(200e9)
        )
    if sys.platform == "win32":
        cache = joblib.Memory(
            "D:\.joblib", verbose=0, compress=True, bytes_limit=int(200e9)
        )
    cache.reduce_size()
else:
    if in_jupyter():
        cache = joblib.Memory(verbose=1, compress=True)
    else:
        cache = Standin()


In [85]:
# | exports


@cache.cache
def attachment_download(href):
    res = rq.get(href)
    res.raise_for_status()
    return DownloadContent(res.content)


In [86]:
# | export


class IncrementalPipeline:
    """
    A class whose instances can dynamically store functions.
    When used as a callable passes each stages retunred object as args into each successive function.
    Results are wrapped if needed.
    """

    def __init__(self, name: str, funcs: List[Callable] = None) -> None:
        self._name = name
        self._funcs = L(funcs)
        self._fuse_cache = None

    def __repr__(self) -> str:
        return f"{IncrementalPipeline.__name__}(name='{self._name}', _funcs={[f.__name__  for f in self._funcs]})"

    def decorate_func(self, func):
        self.append_func(func)
        return func

    def append_func(self, *funcs: List[Callable]):
        self._funcs += funcs
        return self

    def __getitem__(self, idx: Union[int, slice, Iterable]):
        return IncrementalPipeline(self._name, self._funcs[idx])

    def __call__(
        self,
        *args: list[Any],
        tqdm_position: int = 0,
        **init_kwargs: dict[Any, Any],
    ):
        try:
            for i, f in tqdm(
                enumerate(self._funcs), leave=None, position=tqdm_position
            ):
                if not i:
                    res = f(*args, **init_kwargs)
                else:
                    res = f(*res)
                res = res if is_listy(res) else (res,)
        except (ValueError, TypeError) as e:
            raise ValueError(f'{i}: When calling "{f}", args: {args}') from e

        if isinstance(res, (tuple, fastuple)):
            # unwrap redundant tuple
            return res if res.__len__() > 1 else res[0]
        else:
            return res

    @delegates(tqdm)
    def run_group(
        self,
        job_args: Iterable[Tuple[Any]],
        init_kwargs: Dict[str, Any] = None,
        **kwargs,
    ):
        with tqdm(enumerate(job_args), **{**dict(desc="Jobs"), **kwargs}) as ti:
            for _, arg in ti:
                yield self(
                    *arg,
                    **{
                        **dict(
                            init_kwargs=dict() if init_kwargs is None else init_kwargs,
                            tqdm_position=1,
                        ),
                        **kwargs,
                    },
                )


In [87]:
p = IncrementalPipeline("Eg 1")

p


IncrementalPipeline(name='Eg 1', _funcs=[])

In [88]:
p = IncrementalPipeline("Eg 2")


@p.decorate_func
def add_one(a, **_):
    return a + 1


p.append_func(add_one)
p


IncrementalPipeline(name='Eg 2', _funcs=['add_one', 'add_one'])

In [89]:
p(1)


2it [00:00, 14563.56it/s]


3

In [90]:
L(range(10)), L(p[:1].run_group(L(range(10)).map(fastuple)))


Jobs: 10it [00:00, 79.99it/s]


((#10) [0,1,2,3,4,5,6,7,8,9], (#10) [1,2,3,4,5,6,7,8,9,10])

In [91]:
# | hide
import nbdev

nbdev.nbdev_export()
